In [1]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from sympy import Point, Polygon 
import math
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
import os
import uvicorn
from threading import Thread
from io import BytesIO
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
#Feed-ul de la camera
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cam.isOpened():
        ret, frame = cam.read()

        #colorarea frame-ului din BGR in RGB
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False

        #detectarea modelului
        pose_model = pose.process(img)
        
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow("Camera Feed", img)
        if cv2.waitKey(1) == ord('q'):
            break
            
    cam.release()
    cv2.destroyAllWindows()

In [3]:
def calcul_unghi_2_drepte(a, b, c, d):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    d = np.array(d)
    
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi



In [4]:
def calcul_unghi_orizont(a, b):
    a = np.array(a)
    b = np.array(b)
    c = [0, 0.5, 0]
    d = [0.5, 0.5, 0]
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [5]:
def prelucrare_img(nume_imagine, path_salvare, tip):
    global factor_scalare
    global pose_model
    
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    
        img = cv2.imread(nume_imagine)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
    
        #detectarea modelului
        pose_model = pose.process(img)
            
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        factor_scalare = int(img.shape[0] / 720)
        if tip == 1:
            img = adaug_linii_suport_front(img)
            img_al = afisare_aliniament(img, mijloace, 1);
        else:
            img = adaug_linii_suport_profil(img)
            img_al = afisare_aliniament(img,  puncte_de_interes, 2);

        copie = img.copy()
        for lin in range(0, img.shape[1], 50 * factor_scalare):
            cv2.line(copie, (lin, 0), (lin, img.shape[0]), (200, 200, 200), 1)

        for lin in range(0, img.shape[0], 50 * factor_scalare):
            cv2.line(copie, (0, lin), (img.shape[1], lin), (200, 200, 200), 1)

        img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
        img_al = cv2.addWeighted(img_al, 0.8, copie, 0.2, 0)
        cv2.imwrite('img_prelucrata.jpg', img)

        cv2.imwrite(path_salvare, img_al)

        

In [6]:
def prelucrare_img(img, path_salvare, tip):
    global factor_scalare
    global pose_model
    
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
    
        #detectarea modelului
        pose_model = pose.process(img)
            
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        factor_scalare = int(img.shape[0] / 720)
        if tip == 1:
            img = adaug_linii_suport_front(img)
            img_al = afisare_aliniament(img, mijloace, 1);
        else:
            img = adaug_linii_suport_profil(img)
            img_al = afisare_aliniament(img,  puncte_de_interes, 2);

        copie = img.copy()
        for lin in range(0, img.shape[1], 50 * factor_scalare):
            cv2.line(copie, (lin, 0), (lin, img.shape[0]), (200, 200, 200), 1)

        for lin in range(0, img.shape[0], 50 * factor_scalare):
            cv2.line(copie, (0, lin), (img.shape[1], lin), (200, 200, 200), 1)

        img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
        img_al = cv2.addWeighted(img_al, 0.8, copie, 0.2, 0)
        cv2.imwrite('img_prelucrata.jpg', img)

        cv2.imwrite(path_salvare, img_al)


In [7]:
def setup_api():
    app = FastAPI()

    UPLOAD_FOLDER = os.getcwd()
    PROCESSED_FOLDER = "processed"
    os.makedirs(UPLOAD_FOLDER, exist_ok=True)
    os.makedirs(PROCESSED_FOLDER, exist_ok=True)

    app.add_middleware(
        CORSMiddleware,
        allow_origins = ["*"],  # Permite cereri din orice sursă
        allow_credentials = True,
        allow_methods = ["*"],  # Permite toate metodele HTTP (GET, POST etc.)
        allow_headers= ["*"],  # Permite toate header-ele
    )

    app.mount("/processed", StaticFiles(directory="processed"), name="processed")
    
    @app.post("/upload/")
    async def incarcare_imagine(file : UploadFile = File(...)):
        img_path = f"{UPLOAD_FOLDER}/{file.filename}"
        output_path = f"{PROCESSED_FOLDER}/processed_{file.filename}"

        data_img = await file.read()

        nparr = np.frombuffer(data_img, np.uint8)  # Convertește datele binare într-un array NumPy
        img_cv = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        #with open(img_path, "wb") as f:
            #f.write(await file.read())
        prelucrare_img(img_cv, output_path, 1)
        return {"processed_image": f"/processed/processed_{file.filename}"}

    @app.get("/processed/{filename}")
    async def get_imagine_prelucrata(filename: str):
        return FileResponse(f"{PROCESSED_FOLDER}/processed_{filename}")

    return app
    

In [8]:
def adaug_text(img, coord_y, denumire, unghi):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype("arial.ttf", 20 * factor_scalare)
    
    box = draw.textbbox((0, 0), denumire, font)
    lungime_text = box[2] - box[0]
    inaltime_text = box[3] - box[1]
    draw.rectangle([(8, coord_y - inaltime_text - 16 * factor_scalare), (8 + lungime_text + 2 * factor_scalare, coord_y - 8 * factor_scalare)], (255, 255, 255))
    draw.text((10, coord_y - 2 *inaltime_text), denumire, (0, 0, 0), font)

    box = draw.textbbox((0, 0), unghi, font)
    lungime_text = box[2] - box[0]
    inaltime_text = box[3] - box[1]
    draw.rectangle([(img.shape[1] - 8 - lungime_text, coord_y - inaltime_text - 12 * factor_scalare), (img.shape[1] - 8, coord_y - 8 * factor_scalare)], (255, 255, 255))
    draw.text((img.shape[1] - 6 - lungime_text, coord_y - 2*inaltime_text), unghi, (0, 0, 0), font)

    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    return img

In [9]:
def adaug_linii_suport_front(img):
    global mijloace
    global vect_unghiuri_front
    try:
            puncte = pose_model.pose_landmarks.landmark
            puncte_metri = pose_model.pose_world_landmarks.landmark
            
    except:
            pass
   
    print(len(puncte_metri))
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]
    denumiri = ["Urechi", "Umeri", "Solduri", "Genunchi", "Glezne"]
    mijloace = []
    vect_unghiuri_front = []

    coord_sold_dr_metri = (puncte_metri[mp_pose.PoseLandmark.RIGHT_HIP.value].x, puncte_metri[mp_pose.PoseLandmark.RIGHT_HIP.value].y, puncte_metri[mp_pose.PoseLandmark.RIGHT_HIP.value].z)
    print(masuratori_antropometrice(puncte_metri, puncte_linii_suport_dr, puncte_linii_suport_st))
    
    for (puncte_cheie_dr, puncte_cheie_st, denumire) in zip(puncte_linii_suport_dr, puncte_linii_suport_st, denumiri) :
        punct11 = (int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]), int(puncte[puncte_cheie_dr.value].z))
        punct12 = (int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]), int(puncte[puncte_cheie_st.value].z))

            
        cv2.line(img, punct11[:-1], punct12[:-1], (255, 255, 255), factor_scalare)
        cv2.circle(img, punct11[:-1], 10, (0, 0, 0), factor_scalare)
        cv2.circle(img, punct12[:-1], 10, (0, 0, 0), factor_scalare)
        coord_y = int(((puncte[puncte_cheie_dr.value].y + puncte[puncte_cheie_st.value].y)/2) * img.shape[0])
        coord_x = int(((puncte[puncte_cheie_dr.value].x + puncte[puncte_cheie_st.value].x)/2) * img.shape[1])
        mijloace.append((coord_x, coord_y));
        punct1 = (0, coord_y)
        punct2 = (img.shape[1], coord_y)
        cv2.line(img, punct1, punct2, (123, 45, 200), factor_scalare)
        unghi = round(calcul_unghi_orizont(punct11, punct12), 2)
        vect_unghiuri_front.append(unghi)
        unghi_str = str(unghi) + "°"
        img = adaug_text(img, coord_y, denumire, unghi_str)
       
        
    copie = img.copy()
    for i, puncte_cheie_dr in enumerate(puncte_linii_suport_dr[1:-1], 1):
            punct11 = (int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]))
            puncte_cheie_next = puncte_linii_suport_dr[i + 1]
            punct12 = (int(puncte[puncte_cheie_next.value].x * img.shape[1]), int(puncte[puncte_cheie_next.value].y * img.shape[0]))
            cv2.line(copie, punct11, punct12, (255, 255, 255), 2 * factor_scalare)
          
    for i, puncte_cheie_st in enumerate(puncte_linii_suport_st[1:-1], 1):
            punct11 = (int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]))
            puncte_cheie_next = puncte_linii_suport_st[i + 1]
            punct12 = (int(puncte[puncte_cheie_next.value].x * img.shape[1]), int(puncte[puncte_cheie_next.value].y * img.shape[0]))
            cv2.line(copie, punct11, punct12, (255, 255, 255), 2 * factor_scalare)

    img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)

    return img

In [10]:
def calcul_scalare_metrii_pixeli(coord_sold_dr_metri, coord_sold_dr, coord_sold_st):
    coord_x_mijloc = int(coord_sold_dr[0] + coord_sold_st[0])/2
    coord_y_mijloc = int(coord_sold_dr[1] + coord_sold_dr[1])/2
    distanta_pixeli = math.sqrt((coord_x_mijloc - coord_sold_dr[0]) ** 2 + (coord_y_mijloc - coord_sold_dr[1]) ** 2)

    distanta_metri = math.sqrt( coord_sold_dr_metri[0] ** 2 +  coord_sold_dr_metri[1]** 2)
    scalare = distanta_metri / distanta_pixeli

    return scalare
    # SA NU UIT sa schimb indexarea cu [4] ca arata ca dracu

In [11]:
def masuratori_antropometrice(puncte_metri, puncte_linii_suport_dr, puncte_linii_suport_st):
    masuratori = {}
    coord_pct_suport_dr = []
    coord_pct_suport_st = []
    for (puncte_cheie_dr, puncte_cheie_st) in zip(puncte_linii_suport_dr, puncte_linii_suport_st):
            coord_pct_suport_dr.append((puncte_metri[puncte_cheie_dr.value].x, puncte_metri[puncte_cheie_dr.value].y, puncte_metri[puncte_cheie_dr.value].z))
            coord_pct_suport_st.append((puncte_metri[puncte_cheie_st.value].x, puncte_metri[puncte_cheie_st.value].y, puncte_metri[puncte_cheie_st.value].z))
            
    mijloc_umeri = ((coord_pct_suport_dr[1][0] + coord_pct_suport_st[1][0])/2, (coord_pct_suport_dr[1][1] + coord_pct_suport_st[1][1])/2, (coord_pct_suport_dr[1][2] + coord_pct_suport_st[1][2])/2)
    mijloc_solduri = ((coord_pct_suport_dr[2][0] + coord_pct_suport_st[2][0])/2, (coord_pct_suport_dr[2][1] + coord_pct_suport_st[2][1])/2, (coord_pct_suport_dr[2][2] + coord_pct_suport_st[2][2])/2)

    masuratori["lungime_trunchi"] = calc_distanta(mijloc_umeri, mijloc_solduri)
    masuratori["femur_stang"] = calc_distanta(coord_pct_suport_st[2], coord_pct_suport_st[3])
    masuratori["femur_drept"] = calc_distanta(coord_pct_suport_dr[2], coord_pct_suport_dr[3])
    masuratori["tibie_stanga"] = calc_distanta(coord_pct_suport_st[3], coord_pct_suport_st[4])
    masuratori["tibie_dreapta"] = calc_distanta(coord_pct_suport_dr[3], coord_pct_suport_dr[4])

    return masuratori

In [12]:
def calc_distanta(pct1, pct2):
    dist = math.sqrt((pct1[0] - pct2[0]) ** 2 + (pct1[1] - pct2[1]) ** 2 + (pct1[2] - pct2[2]) ** 2)
    return dist

In [13]:
def afisare_aliniament(img, coord, tip):
    copie_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(copie_pil)
    
    (_, y_urechi) = coord[0]
    (x_glezne, _) = coord[4]
        
    for i, (x, y) in enumerate(coord[:-1]):
        urm_x, urm_y = coord[i+1]
        pct_filler = [(x, y), (urm_x, urm_y), (x_glezne, urm_y), (x_glezne, y)]
        if (x > x_glezne and urm_x < x_glezne) or (x < x_glezne and urm_x > x_glezne):
            y_int = intersectie(x_glezne, y, x_glezne, urm_y, x, y, urm_x, urm_y)
            draw.polygon([(x, y), (x_glezne, y_int), (x_glezne, y)], (255, 0, 0))
            draw.polygon([(urm_x, urm_y), (x_glezne, y_int), (x_glezne, urm_y)], (255, 0, 0))
        else:
            draw.polygon(pct_filler, (255, 0, 0))

        if tip == 1:
            cv2.line(img, (x, y), (urm_x, urm_y), (255, 0, 0), factor_scalare)
            cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)
    if tip == 1:
        x, y = coord[-1]
        cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)
    copie = cv2.cvtColor(np.array(copie_pil), cv2.COLOR_RGB2BGR)
    img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
    cv2.line(img, coord[4], (x_glezne, y_urechi), (0, 255, 0), factor_scalare)
    return img

In [14]:
def intersectie(x11, y11, x12, y12, x21, y21, x22, y22):
    y_int = int(((x11 * y12 - y11 * x12) * (y21 - y22) - (y11 - y12) * (x21 * y22 - y21 * x22)) / ((x11 - x12) * (y21 - y22) - (y11 - y12) * (x21 - x22)))
    return y_int

In [15]:
def afla_orientare(puncte):
    sold_stg = puncte.landmark[mp_pose.PoseLandmark.LEFT_HIP.value].z
    sold_drept = puncte.landmark[mp_pose.PoseLandmark.RIGHT_HIP.value].z

    if sold_stg > 0:
        return "stanga"
    return "dreapta"
    

In [16]:
def adaug_linii_suport_profil(img):
    global puncte_de_interes
    global vect_unghiuri_profil
    try:
            puncte = pose_model.pose_landmarks
    except:
            pass


    puncte_de_interes = []
    vect_unghiuri_profil = []
    denumiri = ["Flexia cervicală", "Flexia șoldului", "Flexia genunchiului"]
    orientare = afla_orientare(puncte)
    if orientare == "stanga":
        puncte_linii_suport = [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]
    else:
        puncte_linii_suport = [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE]

    copie = img.copy()
    for i, punct in enumerate(puncte_linii_suport[:-1]):
        coord_pct = (int(puncte.landmark[punct.value].x * img.shape[1]), int(puncte.landmark[punct.value].y * img.shape[0]), int(puncte.landmark[punct.value].z))
        punct_urm = puncte_linii_suport[i+1]
        coord_pct_urm = (int(puncte.landmark[punct_urm.value].x * img.shape[1]), int(puncte.landmark[punct_urm.value].y * img.shape[0]), int(puncte.landmark[punct_urm.value].z))
        cv2.line(img, coord_pct[:-1], coord_pct_urm[:-1], (255, 255, 255), factor_scalare)
        cv2.circle(img, coord_pct[:-1], 10, (0, 0, 0), factor_scalare)

        if i:
            (_, coord_y, _) = coord_pct
            punct1 = (0, coord_y)
            punct2 = (img.shape[1], coord_y)
            cv2.line(img, punct1, punct2, (123, 45, 200), factor_scalare)
        
        puncte_de_interes.append(coord_pct[:-1])

    puncte_de_interes.append(coord_pct_urm[:-1])
    cv2.circle(img, coord_pct_urm[:-1], 10, (0, 0, 0), factor_scalare)

    for denumire, (i, _) in zip(denumiri, enumerate(puncte_de_interes[1:4])):
        pct1 = puncte_de_interes[i]
        pct2 = puncte_de_interes[i+1]
        pct3 = puncte_de_interes[i + 2]
        unghi = round(180 - calcul_unghi_2_drepte(pct1, pct2, pct2, pct3), 2) #calculeaza unghiul dintre dreptele formate de (pct1, pct2) cu (pct2, pct3)  
        vect_unghiuri_profil.append(unghi)

        unghi_str = str(unghi) + "°"
        (_, coord_y) = pct2
        img = adaug_text(img, coord_y, denumire, unghi_str)

    return img
         
         

In [17]:

app = setup_api()


In [18]:
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

INFO:     Started server process [12968]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


33
{'lungime_trunchi': 0.47694715190504133, 'femur_stang': 0.41706387312980225, 'femur_drept': 0.4098359956733043, 'tibie_stanga': 0.36990493117452156, 'tibie_dreapta': 0.3837442643674576}
INFO:     127.0.0.1:61649 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:61650 - "GET /processed/processed_img_test8.jpeg HTTP/1.1" 200 OK
33
{'lungime_trunchi': 0.47694715190504133, 'femur_stang': 0.41706387312980225, 'femur_drept': 0.4098359956733043, 'tibie_stanga': 0.36990493117452156, 'tibie_dreapta': 0.3837442643674576}
INFO:     127.0.0.1:61682 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:61683 - "GET /processed/processed_img_test8.jpeg HTTP/1.1" 200 OK
33
{'lungime_trunchi': 0.49498015771155063, 'femur_stang': 0.40552302816579866, 'femur_drept': 0.40206021833281846, 'tibie_stanga': 0.3912983152158017, 'tibie_dreapta': 0.37056127028296926}
INFO:     127.0.0.1:61718 - "POST /upload/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:61719 - "GET /processed/processed_img_test7.jpeg HTTP/1.1" 2